In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, concat

spark_session = SparkSession.builder \
    .appName("silver_layer") \
    .config("spark.databricks.delta.autoCompact.enabled", "true") \
    .config('spark.sql.parquet.vorder.enabled', 'true') \
    .config("delta.autoOptimize.optimizeWrite", "true")
    .getOrCreate()

def table_names(spark_session):
    tables= spark_session.sql("SHOW TABLES")
    table_names= [table["tableName"] for table in tables.collect()]
    return table_names

table_list_withBronze= table_names(spark_session)

silver_name= 'silver_Medicare_CMS_PartD'

def silver_update(spark_session, table_list_withBronze):

    for tab in table_list_withBronze:
        updated_rows = spark_session.sql(
            f"""
            ALTER TABLE {tab} SET TBLPROPERTIES (delta.tuneFileSizesForRewrites = true)
            MERGE INTO {silver_name} AS target
            USING {tab}  AS source
            ON target.ingestion_timestamp = source.ingestion_timestamp AND source.ingestion_timestamp IS NOT NULL
            WHEN NOT MATCHED THEN INSERT *;"""

        )
        print(f"{updated_rows.count()} rows have been updated in {silver_name} using {tab}.")


def silver_first_load(spark_session, table_list, silver_name):

    for i in table_list_withBronze:
        df = spark_session.read.format('delta').table(i)

        df = df.withColumn("Prscrbr_City_State", concat(df.Prscrbr_City, lit(", "), df.Prscrbr_State_Abrvtn)) \
                .withColumn("Prscrbr_Full_Name", concat(df.Prscrbr_Last_Org_Name, lit(", "), df.Prscrbr_First_Name)) \
                
        df.write.mode("append").format("delta").save(f"Tables/{silver_name}")
        #df.write.mode("append").format("delta").save(f"Tables/{silver_name}")

if silver_name in table_list_withBronze:
    silver_update(spark_session, table_list_withBronze)
    
else: silver_first_load(spark_session, table_list_withBronze, silver_name)

StatementMeta(, a33c509c-7989-4b1f-b406-756593935f15, 4, Finished, Available)

In [3]:
from pyspark.sql.functions import col
df= spark.read.format('delta').option('inferschema','true').option('headers','true').load('Tables/silver_Medicare_CMS_PartD')
df1= df.groupBy('Year').count()

display(df1)

StatementMeta(, a33c509c-7989-4b1f-b406-756593935f15, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, f5979be2-b8c1-4618-978c-18e033f271bd)